## Cabeçalho

In [ ]:
from pathlib import Path

import yaml
from ckanapi import RemoteCKAN
from IPython.display import clear_output

In [ ]:
%cd /home/vncsna/Base

In [ ]:
# LEMBRAR QUE O NOME DA BASE FOI MUDADA EM
# bases/br_anatel_banda_larga_fixa
# -> bases/acessos_de_banda_larga_fixa_no_brasil
# bases/br_anatel_telefonia_movel
# -> bases/acessos_de_telefonia_movel_no_brasil
# bases/br_inep_sinopse_estatistica_educacao_basica
# -> bases/sinopses_estatisticas_da_educacao_basica

## Atualização da Descrição

In [ ]:
# chaves de api omitidas

ckan_local = RemoteCKAN("http://localhost:5000/")
ckan_prod = RemoteCKAN("https://basedosdados.org/")
ckan_stage = RemoteCKAN("https://staging.basedosdados.org/")

In [ ]:
def get_site_notes(name: str, ckan: RemoteCKAN = ckan_prod):
    try:
        return ckan.action.package_show(name_or_id=name)["notes"]
    except:
        return ""


def get_mais_notes(name: str, folder: str = "/home/vncsna/Base/mais/bases"):
    name = name.replace("-", "_")
    filepath = Path(f"{folder}/{name}/dataset_config.yaml")

    if filepath.exists():
        with open(filepath, "r") as file:
            package = yaml.safe_load(file.read())
            return package["description"]

    return ""


def update_notes_one(name: str, ckan: RemoteCKAN = ckan_stage):
    try:
        package = ckan.action.package_show(name_or_id=name)

        mais_notes = get_mais_notes(name)
        site_notes = get_site_notes(name)

        if len(site_notes) > 0:
            package["notes"] = site_notes
        else:
            package["notes"] = mais_notes

        package.pop("dataset_id", 0)
        package.pop("dataset_args", 0)
        package.pop("download_type", 0)
        if "extras" in package:
            if len(package["extras"]) > 0:
                if "value" in package["extras"][0]:
                    package["extras"][0]["value"].pop("download_type", 0)

        return ckan.action.package_update(**package)
    except:
        print(name)


def update_notes_all(ckan: RemoteCKAN = ckan_stage):
    packages = ckan.action.package_list()

    for package in packages:
        update_notes_one(package)

In [ ]:
update_notes_all()

## Debugando em um CKAN Local

In [ ]:
# # debugging on local ckan
# # update local ckan with stage packages
# for package in ckan_stage.action.package_list():
#   package = ckan_stage.action.package_show(name_or_id=package)

#   package.pop("download_type", 0)

#   if "extras" in package:
#     if len(package["extras"]) > 0:
#       if "value" in package["extras"][0]:
#         package["extras"][0]["value"].pop("download_type", 0)

#   package.pop("id", 0)
#   package.pop("dataset_id", 0)
#   package.pop("dataset_args", 0)

#   for index, resource in enumerate(package.get("resources", [])):
#     package["resources"][index].pop("id", 0)

#   ckan_local.action.package_create(**package)